# Symmetrical choiceset:

We want to symmetrically sample both sides of the subjective equality point. 
That means having equal amount of trials in each side of the SV difference. 

This version of the code doesn't use python external libraries. The reason is to optimize the code to make it compatible with online based tasks that use Js. 

JS script data types we are going to use:
- Arrays: written with square brackets. Items are separated by commas. This is analogous to python lists. 

    ``` const cars = ["Saab", "Volvo", "BMW"]; ```
- Objects: JavaScript objects are written with curly braces {}.

Object properties are written as name:value pairs, separated by commas.

This is analogous to python dictionaries. I intend to use them to create the desired choicet. 

        const person = {firstName:"John", lastName:"Doe",  age:50, eyeColor:"blue"}

In [125]:
import csv #This is the only library I will use. However, it is built in python. 
post_mean_gain = [0.45456, 1.0343]
post_mean_loss = [0.896596, -0.765]

The idea is to create an output CS without using pandas :( 
    We are going to do this by using lists and dicitonaries. The idea is to creaate the trials as lists of dictionries

Trials = 
```  trials = 
[{'p_reward': 0.13,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 0.7724887228000991,
  'SV_New_Safe': 0.38624436140004953,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 0.12390278661966936},
 {'p_reward': 0.25,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 1.4855552361540365,
  'SV_New_Safe': 0.7427776180770183,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 0.5206065839498941},
 {'p_reward': 0.38,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 2.2580439589541355,
  'SV_New_Safe': 1.1290219794770677,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 1.3052453010492082},
 {'p_reward': 0.5,
...
  'ambiguity': 0.0,
  'SV_reward': 4.4566657084621095,
  'SV_New_Safe': 2.2283328542310548,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 5.806103319838627}]
```
With that, we can create the CSV at the end


# 1. Input: ADO parameters

In [126]:
alpha_pos, beta_pos = post_mean_gain[0], post_mean_gain[1] #parameters for gains
alpha_neg, beta_neg = post_mean_loss[0], post_mean_loss[1] #parameters for losses

In [127]:
#Declare the variables
p_options = [0.75, 0.5, 0.38, 0.25, 0.13]
amb_options = [0.24, 0.5, 0.74]
A_null = 0.0 # no ambiguity
Vmax = 50.0 #The maximum amount we can pay to participants
Vmin = -50.0 #The minimum amount we can take from participants
Vsafe_pos = 5.0 # The old safe option in positive trials
Vsafe_neg = -5.0 # The old safe option in negative trials

#First, we create an empty dictionary that we will populate with the trials info. 
empty_dict = {'p_reward': None,
  'value_lottery': None,
  'category': None,
  'ambiguity': None,
  'SV_lottery':None,
  'value_safe': None,
  'SV_safe': None
  }

In [133]:
##FUNCTIONS##

#Create the extreme positive gains and losses
def get_extreme(p_options, 
                domain='gain'): 
    ###Obtains extreme gain/loss for each probability
    """
    Inputs: 
        - p_options = list of probability values (float)
    Output:
        - dict containing highest possible gain or loss for each probability ($50 or -$50)
    """
    extr = {}
    V = Vmax #default gain domain (max gain)
    if domain == 'loss': #loss domain 
        V = Vmin #assign min value (max loss)
    for p in p_options: #iterate through probability levels
        extr[p] = V  #assign max value (gain) or min value (loss) for each probability
    return extr

def add_ambig_ext(extrdict, amb_options, Vmax=50, prob_amb = 0.5, trials = 'gains'):
    amb_dict = {prob_amb: amb_options}
    print(amb_dict)

add_ambig_ext(extrpos, amb_options)

{0.5: [0.24, 0.5, 0.74]}


In [136]:
##GAINS
extrpos = empty_row.copy()
extrpos = get_extreme(p_options)
print(extrpos)
def get_extr_rows(extr_dict, domain='gain'):
    app = []
    for p, value in extr_dict.items():
        app.append({'p_reward': p, 'value_reward': value})
    category = 'Extr_pos' if domain == 'gain' else 'Extr_neg'
    for row in app:
        row['category'] = category
        row['ambiguity'] = A_null
    return app

rows_pos = get_extr_rows(extrpos)
rows_pos


{0.75: 50.0, 0.5: 50.0, 0.38: 50.0, 0.25: 50.0, 0.13: 50.0}


[{'p_reward': 0.75,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0},
 {'p_reward': 0.5,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0},
 {'p_reward': 0.38,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0},
 {'p_reward': 0.25,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0},
 {'p_reward': 0.13,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0}]

In [137]:
##LOSSES
extrneg = empty_dict.copy()
extrneg = get_extreme(p_options, domain ='loss')
print(extrneg)

rows_neg = get_extr_rows(extrneg, domain ='loss')
rows_neg

{0.75: -50.0, 0.5: -50.0, 0.38: -50.0, 0.25: -50.0, 0.13: -50.0}


[{'p_reward': 0.75,
  'value_reward': -50.0,
  'category': 'Extr_neg',
  'ambiguity': 0.0},
 {'p_reward': 0.5,
  'value_reward': -50.0,
  'category': 'Extr_neg',
  'ambiguity': 0.0},
 {'p_reward': 0.38,
  'value_reward': -50.0,
  'category': 'Extr_neg',
  'ambiguity': 0.0},
 {'p_reward': 0.25,
  'value_reward': -50.0,
  'category': 'Extr_neg',
  'ambiguity': 0.0},
 {'p_reward': 0.13,
  'value_reward': -50.0,
  'category': 'Extr_neg',
  'ambiguity': 0.0}]

# Output CSV

In [139]:
# Extract column names from the keys of the first dictionary
fieldnames = crdm_trials[0].keys()

# Define the name of the CSV file
csv_filename = 'crdm_trials.csv'

# Write the data to the CSV file
with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Write the data rows
    for row in crdm_trials:
        writer.writerow(row)

print(f"CSV file '{csv_filename}' has been created.")

NameError: name 'trials' is not defined